#### Goal - show how the piping in coppertop makes it easier to do non-trivial computations in Excel doing a PCA on some equities as an example

I'm going to show my workings here - for didactic and requirements reasons


OPEN: groot needs explaining (add an issue)

In [1]:
%matplotlib inline

import datetime, polars as pl, matplotlib.pyplot as plt, pandas as pd

from coppertop.pipe import *
from bones.core.sentinels import Missing
import dm.core, dm.pp, dm.polarframe, dm.pandaframe, dm.fs, dm.linalg.core
from _ import *
from _ import dm, fs

<br>

#### piped function calls vs nested function calls

Many people seem to find the left to right style of pipeline style easier to read than nesting functions. Compare this:

In [2]:
list(filter(lambda x: x > 2, map(lambda x: x + 1, [1,2,3])))

with this (where map => collect, filter => select):

In [3]:
[1,2,3] >> collect >> (lambda x: x + 1) >> select >> (lambda x: x > 2)

or using the convention that a trailing underscore in a function names indicates a deferred function:

In [4]:
@coppertop(style=binary)
def collect_(xs, fn):
    return map(fn, xs)

@coppertop(style=binary)
def select_(xs, fn):
    return filter(fn, xs)

[1,2,3] >> collect_  >> (lambda x: x + 1) >> select_ >> (lambda x: x > 2) >> to >> list

Just for interest these are the bones equivalents
```
(1,2,3) collect {x + 1} select {x > 2}
(1,2,3) collect_ {x + 1} select_ {x > 2} to <:N**count>

```


<br>

#### PCA

First - on a train journey I downloaded some stocks from Yahoo Finance - https://finance.yahoo.com/quote/MSFT/history?p=MSFT etc




In [5]:
fs >> DD

In [6]:
'./analysis/data' >> fs.scanFiles

In [7]:
'./analysis/data/AMZN.csv' >> dm.polars.csv.read

or as a pandas dataframe

In [8]:
'./analysis/data/AMZN.csv' >> dm.polars.csv.read >> to >> pd.DataFrame

In [9]:
d1 = datetime.date(2021, 12, 1)
d2 = datetime.date(2022, 11, 30)

AMZN = './analysis/data/AMZN.csv'  \
    >> dm.polars.csv.read \
    >> take >> ['Date', 'Close'] \
    >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))
MSFT = './analysis/data/MSFT.csv'  \
    >> dm.polars.csv.read \
    >> take >> ['Date', 'Close'] \
    >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))
AAPL = './analysis/data/AAPL.csv'  \
    >> dm.polars.csv.read \
    >> take >> ['Date', 'Close'] \
    >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))
TSLA = './analysis/data/TSLA.csv'  \
    >> dm.polars.csv.read \
    >> take >> ['Date', 'Close'] \
    >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))
GOOG = './analysis/data/GOOG.csv'  \
    >> dm.polars.csv.read \
    >> take >> ['Date', 'Close'] \
    >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))

Let's join them together. I have aj (asof join for dataframes)

OPEN: aj needs to name the columns better - the standard polars ones are not scalable and the api should be the same accross all versions, maybe a kdb like column rename would be better.

OPEN: naming is slightly different for immutable languges - withNames for mutable languages, setNames for immutable - is `MSFT >> setNames >> [_,'MSFT']` good? q had xcols and xcol and I had to look up, `MSFT >> reorder >> ['MSFT']`


In [10]:
aj >> sig

OPEN: alias polars.internals.dataframe.frame.DataFrame as something clear and short \
let's try this (turns out to be a good guess but don't like the new name `Close_right`)

In [11]:
AMZN >> aj(_,_,'Date', 'Date') >> MSFT 

as I have to know that the new column name will be appended with "\_right" and that if I call it again it throws an error - to me the api just doesn't scale

In [12]:
try:
    AMZN \
        >> aj(_,_,'Date', 'Date') >> MSFT \
        >> aj(_,_,'Date', 'Date') >> AAPL 
except Exception as ex:
    ex >> PP

In [13]:
AMZN >> rename >> 'Close' >> 'AMZN'  \
    >> aj(_,_,'Date', 'Date') >> MSFT >> rename >> 'Close' >> 'MSFT' \
    >> aj(_,_,'Date', 'Date') >> AAPL >> rename >> 'Close' >> 'AAPL' 

In [14]:
closesFrame = AMZN >> rename >> 'Close' >> 'AMZN' \
    >> aj(_,_,'Date', 'Date') >> MSFT >> rename >> 'Close' >> 'MSFT' \
    >> aj(_,_,'Date', 'Date') >> AAPL >> rename >> 'Close' >> 'AAPL' \
    >> aj(_,_,'Date', 'Date') >> TSLA >> rename >> 'Close' >> 'TSLA' \
    >> aj(_,_,'Date', 'Date') >> GOOG >> rename >> 'Close' >> 'GOOG'
closesFrame

We can do better

In [15]:
d1 = datetime.date(2021, 12, 1)
d2 = datetime.date(2022, 11, 30)

allSeries = ['AMZN', 'MSFT', 'AAPL', 'TSLA', 'GOOG'] >> collect >> (lambda name: 
    f'./analysis/data/{name}.csv'
        >> dm.polars.csv.read 
        >> take >> ['Date', 'Close']
        >> rename >> 'Close' >> name
        >> select >> ((d1 <= pl.col("Date")) & (pl.col("Date") <= d2))
) 

closesFrame = allSeries >> inject(_, Missing, _) >> (lambda prior, each: 
    each if prior is Missing else prior >> aj(_,_,'Date','Date') >> each
)

closesFrame

<br>

next take diffs and do the SVD (i.e. to generated the PCA loadings, aka eigenvectors, etc)

In [17]:
vs = closesFrame >> drop >> 'Date' >> takePanel >> diffRows>> dm.linalg.pca >> withKeys >> ['v','s']
v, s = vs >> values

In [18]:
plt.plot(v >> atCol >> 0, 'red')
plt.plot(v >> atCol >> 1, 'orange')
plt.plot(v >> atCol >> 2, 'green')
plt.plot(v >> atCol >> 3, 'blue')
plt.plot(v >> atCol >> 4, 'black')

<br>

#### Putting it in Excel using pyxll

some code to call an anonymous python function via pyxll:

In [ ]:
import traceback, sys

@xl_func(auto_resize=True)
@xl_arg("srcOrFn", "union<str[], str>")
def pyFn(srcOrFn, *args):
    if isinstance(srcOrFn, list):
        srcOrFn = '\n'.join(srcOrFn)
    fn = eval(srcOrFn, globals())
    try:
        answer = fn(*args)
    except Exception as ex:
        et, ev, tb = sys.exc_info()
        lines = ['PYTHON TRACEBACK']
        for e in traceback.format_exception(et, ev, tb):
            lines.extend(e.split('\n'))
        return lines
    return answer


doing the svd in exel:

```
=pyFn("lambda x: x >> drop >> 'Date' >> takePanel >> diffRows>> dm.linalg.pca >> withKeys >> ['v','s']", {cell with closesFrame object})
```